#### Importing relavent libraries

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime #make logs on tensorboard based on datetime

#### Data preprocessing

In [3]:
#loading the data
mnist_data, mnist_info = tfds.load('mnist',as_supervised= True, with_info=True)

In [4]:
#defining the hyperparams
BUFFER_SIZE = 70_000
NUM_EPOCHS = 20
BATCH_SIZE = 128

In [5]:
mnist_train, mnist_test = mnist_data['train'],mnist_data['test']

In [8]:
# scale func
def scale(image , label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image,label

In [9]:
#normalizing the datasets
train_val_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [10]:
#defining the validation size
val_size = tf.cast(.1 * mnist_info.splits['train'].num_examples, tf.int64)

In [11]:
#determining the test size
test_size = tf.cast(mnist_info.splits['test'].num_examples, tf.int64)

In [12]:
#shuffling the data
train_val_data = train_val_data.shuffle(BUFFER_SIZE)

In [13]:
#separatinh train and validation sets
train_data = train_val_data.skip(val_size)
val_data = train_val_data.take(val_size)

In [14]:
#batching the data
train_data = train_data.batch(BATCH_SIZE)
val_data = val_data.batch(val_size)
test_data = test_data.batch(test_size)

#### Defining the CNN model

In [17]:
#outlining the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50,5,activation ='relu', input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50,3, activation ='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [19]:
model.summary(line_length=80)

Model: "sequential"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
conv2d_1 (Conv2D)                   (None, 24, 24, 50)              1300        
________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)      (None, 12, 12, 50)              0           
________________________________________________________________________________
conv2d_2 (Conv2D)                   (None, 10, 10, 50)              22550       
________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)      (None, 5, 5, 50)                0           
________________________________________________________________________________
flatten (Flatten)                   (None, 1250)                    0           
________________________________________________________________________________
dense (D

In [20]:
#defining the loss function and incorporating softmax in it
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [21]:
#compiling the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [22]:
#defining early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='auto',
    min_delta = 0,
    patience=2,
    restore_best_weights=True,
    verbose = 0
)

In [25]:
#logging the training process data
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime('%Y%m%d--%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq= 1)

In [27]:
#training the network
model.fit(
    train_data,
    epochs=NUM_EPOCHS,
    verbose=2,
    callbacks=[tensorboard_callback,early_stopping],
    validation_data = val_data
)

Epoch 1/20
422/422 - 28s - loss: 0.2678 - accuracy: 0.9250 - val_loss: 0.0832 - val_accuracy: 0.9750
Epoch 2/20
422/422 - 23s - loss: 0.0729 - accuracy: 0.9785 - val_loss: 0.0647 - val_accuracy: 0.9788
Epoch 3/20
422/422 - 28s - loss: 0.0552 - accuracy: 0.9826 - val_loss: 0.0385 - val_accuracy: 0.9897
Epoch 4/20
422/422 - 29s - loss: 0.0447 - accuracy: 0.9867 - val_loss: 0.0497 - val_accuracy: 0.9845
Epoch 5/20
422/422 - 62s - loss: 0.0375 - accuracy: 0.9889 - val_loss: 0.0416 - val_accuracy: 0.9865


#### Testing the model

In [28]:
model_loss, model_acc = model.evaluate(test_data)


1/1 [==============================] - 1s 1s/step - loss: 0.0489 - accuracy: 0.9856


In [34]:
print('model loss:{0:.4f}. model accuracy:{1:.2f}%'.format(model_loss, model_acc*100))

model loss:0.0489. model accuracy:98.56%


#### Visualising in Tensorboard

In [35]:
#loading the tensorboard extension
%load_ext tensorboard
%tensorboard --logdir "logs/fit"